In [ ]:
using CSV, DataFrames, Random, LinearAlgebra, Distances, Distributions, SpecialFunctions, Plots, Distributed

addprocs(11)

@everywhere begin
    
using CSV, DataFrames, Random, LinearAlgebra, Distances, Distributions, SpecialFunctions, Plots
    
nsp=20

samples=[collect(0:10:100); collect(100:50:1000)]

#Necessary functions
    
 #Necessary functions

    function alpha_gt(x::Float64, y::Float64, omega::Float64, t::Float64)
    
        al=0
        if(abs(x-y)<t) 
            
             al=exp(-((x-y)^2)/(omega^2))

        end
        return al
    end
    
    function alpha_tri(x::Float64, y::Float64, slope::Float64, t::Float64)
        
        alpha=0.0
        
        if ( abs(x-y) < t) 
            
            alpha=1- (slope*abs(x-y))
            
        end
            
        return max(0,alpha)
        
    end
    
    function mnnds(a::Vector{Float64})
    
    a=filter!(!isnan,a)
    if(length(a)>1)
        a=sort(a,rev=true)
        nnd=zeros(Float64,length(a))
        nnd[1]=a[2]-a[1]
        nnd[length(nnd)]=a[length(a)]-a[length(a)-1]
        for i in 2:((length(a)-1))
            nnd[i]=min((a[i-1]-a[i]),(a[i]-a[i+1]))
        end
        ranges=a[1]-a[end]
        mnnd=sum(nnd)/length(nnd)
        mmax=ranges/(length(a)-1)

        return mnnd/mmax
        
        else return 0.0
        end
        
    end
        
    
    function qgprob(n::Int64)
    
        #All possible phenotypes
        pheno= collect(1:(2*n+1)) ./ (2*n+1)
        nt=length(pheno)
    
        G=zeros(Float64,n+1,n+1,n+1)
    
        for i in 0:n, j in 0:i, k in max(0,(i+j-n)):min(n,(i+j))
                    m=collect(0:min(j,k,i+j-k))
                    G[1+i,1+j,1+k]=sum(pdf.(Hypergeometric(i,n-i,j),m).*pdf.(Binomial.(i+j .- (2 .* m)),k .- m))
        end
    
        for k in 0:n
            G[:,:,1+k]=G[:,:,1+k]+transpose(G[:,:,1+k])
            for i1 in 0:n
                G[i1+1,i1+1,k+1] /= 2
            end
        end
    
        ind_haplR=zeros(Float64,2*n+1, 2*n+1)
    
        for k in 0:n
            for i in 0:n
                 ind_haplR[1+i,1+k] = G[1+i,1,1+k]
                for j in 0:n
                    ind_haplR[1+j+n,1+k]=G[1+n,1+j,1+k]
                end
            end
        end
    
        R=zeros(Float64,nt,nt,nt)
    
        for i in 0:(2*n), j in 0:(2*n), q in 0:(2*n)
             R[1+i,1+j,1+q]= sum(ind_haplR[1+i,1 .+ (0:q)] .*
                                 ind_haplR[1+j,1+q .- (0:q)])
        end
    
        return R
    end
    
    
     function single_sim1(time,r,K1,K2,a1,A,R,Ng0,Npop)
        
            nsp=size(Ng0)[1]
            nt=size(Ng0)[2]
            
            Np0=Ng0 .*Npop
            Ngen=deepcopy(Ng0)
            Np=deepcopy(Np0)
        
            dat=zeros(Float64,time+1,nsp,nt)
    
            dat[1,:,:]=Np
            
            #Start the simulation
            for m in 2:(time+1)
        
                #Determine the extinct species
                Np[findall(sum(Np,dims=2) .< 10),:] .= 0
                Ngen[findall(sum(Np,dims=2) .==0),:] .= 0
    
                if all(sum(Np,dims=2) ==0) 
                    break
                else
    
                    newgen=zeros(Float64,nsp,nt)
    
                    #Reproduction event
                    for i in findall(!iszero,sum(eachcol(Ngen)))
    
                        probs=Ngen[i,:]*Ngen[i,:]'
                    
                        newgen[i,:]=[sum(probs.*R[:,:,x1]) for x1 in 1:nt]
                    
                        newgen[i,:] ./= sum(newgen[i,:])
                        
                    end
    
                    newgen .*= sum(Np,dims=2)
    
                    #Selection event
                
                    if size(A)[1]>1
    
                        for i1 in 1:size(newgen)[1]
                    
                            #This is where we have to make big choices!! a)Whether or not to make conspecific individuals compete the 
                            #same way as heterospecifics. b) Interprete the meaning of carrying capacity. Do different numbers
                            #apply for different species? If yes, what is the interpretation of this type of competition?
                           
                            #Reduction in growth rate due to intraspecific competition (not trait-dependent!). The carrying capacity
                            #here is assumed to be the same for all species.

                            rdash=r[i1] * (1 -(sum(newgen[i1,:])/K2))
                        
                            #Impact of interspecific competition

                            comps=[(a1.*sum((A[x2,:]) .* newgen[1:end .!=i1,:]')) for x2 in 1:nt]
                        
                            Np[i1,:] = newgen[i1,:] + (newgen[i1,:] .* rdash .* (1 .-(comps ./K1)))
                        
                        end
                    
                    end
    
                    Np[findall(Np .<1)] .= 0
                    Ngen= Np ./ sum(Np,dims=2)
                    Ngen[isnan.(Ngen)].=0
                end
                
            dat[m,:,:]=Np
                
            end
        
        return dat
        
    end
    
    
    #Blanket function to create trait and population trajectories
    function getsum(time,r,K1,K2,a1,A,R,Ng0,Npop)
        
        res=single_sim1(time,r,K1,K2,a1,A,R,Ng0,Npop)
        
        tstep=size(res)[1]
        nsp=size(res)[2]
        nt=size(res)[3]
        

        pops=zeros(Float64,tstep,nsp)
        trmeans=zeros(Float64,tstep,nsp)

        for i in 1:nsp

        pops[:,i]=[sum(res[x,i,:]) for x in 1:tstep]
        trmeans[:,i]=[sum(res[x,i,:] .* geno)/sum(res[x,i,:])  for x in 1:tstep]

        end

        return([pops,trmeans])
        
    end
        
    reps=collect(1:30)
    loci=[3,4,5,6,7,8,10,15,20,50]
        
    nsp=20


function alpha_gt(x::Float64, y::Float64, omega::Float64, t::Float64)
      
    al=0
    
    if(abs(x-y)<t)         
         al=exp(-((x-y)^2)/(omega^2))
         #denom=(pi/2)*(erf(t/omega)-erf(-t/omega))
         #al/=denom
    end
    return al
end

function alpha_tri(x::Float64, y::Float64, slope::Float64, t::Float64)
    
    alpha=0.0
    
    if ( abs(x-y) < t) 
        
        alpha=1- (slope*abs(x-y))
        
    end
        
    return max(0,alpha)
    
end

#parameter combinations

reps=collect(1:10)
loci=[2,3,4,5,6,7,8,10,15,20,50]

pars=collect(Iterators.product(reps,loci))

function compsim(par::Tuple{Int64, Int64})
    
    rep=par[1]
    n=par[2]

    geno= collect(range(-1,stop=1,length=2*n+1))
    nt=length(geno)
    
    rands=rand(Uniform(-0.8,0.8),nsp)

    N=zeros(Float64,nsp,nt)

    [N[i,:]=pdf.(Uniform(rands[i]-0.2,rands[i]+0.2),geno) for i in 1:nsp]

    N=N ./ sum(N,dims=2)

    Ng0= N ./ (sum.(eachrow(N)))
    Np0= 1000 .* Ng0 
    r=abs.(rand(Uniform(1,1.1),nsp))
        
    
    #Set up a probability matrix of an offspring having a phenotype x when the parents have the phenotypes v and w resp.
    # This code follows the diploid version of an exact hypergeometric model
    #from Shpak and Kondrashov (1999)
    haplR=zeros(Float64,n+1,n+1,n+1)

    for i in 0:n, j in 0:i, k in 0:min(n,(i+j))
        haplR[1+i,1+j,1+k]=sum(pdf.(Hypergeometric(i,n-i,j),max(0, i+j-n):min(i, j)) .*
                              map(x->pdf(Binomial(i+j-2*x),k-x),collect(max(0, i+j-n):min(i,j))))
    end

    for k in 0:n
        haplR[:,:,1+k]=haplR[:,:,1+k]+transpose(haplR[:,:,1+k])
        for i1 in 0:n
            haplR[i1+1,i1+1,k+1] /= 2
        end
    end

    ind_haplR=zeros(Float64,2*n+1, 2*n+1)

    for k in 0:n
        for i in 0:n
            ind_haplR[1+i,1+k] = haplR[1+i,1,1+k]
            for j in 0:n
                ind_haplR[1+j+n,1+k]=haplR[1+n,1+j,1+k]
            end
        end
    end

    R=zeros(Float64,nt,nt,nt)

    for i in 0:(2*n), j in 0:(2*n), q in 0:(2*n)
        R[1+i,1+j,1+q]= sum(ind_haplR[1+i,1 .+ (0:q)] .* 
                            ind_haplR[1+j,1+q .- (0:q)])
    end
    
    
    omegas=[0.1,0.5]
    ts=[0.2,0.5,2]
    a1s=[0.1,0.5,1]
    kernel=["Gaussian","Triangle"]
    
    result=DataFrame()
        
    for i1 in omegas, i2 in ts, i3 in a1s, i4 in kernel
        
        omega=i1
        t=i2
        a1=i3
        
        #Pre-calculate coefficients of competition between pairs of genotypes

        A1=zeros(Float64,nt,nt)
        A0=zeros(Float64,nt,nt)
        
        for j1 in 1:nt, j2 in 1:nt
                
            if i4=="Gaussian"
    
            A0[j1,j2]=alpha_gt(geno[j2],geno[j1],omega,t)
                    
                elseif i4=="Triangle"
                    
            A0[j1,j2]=alpha_tri(geno[j2],geno[j1],omega,t)
                end
            end
                    
        
        A0= A0 ./ 10000        
        A1= A0 .*a1 
        
        
        Ngen=deepcopy(Ng0)
        Np=deepcopy(Np0)
        
        #Start the simulation
        for m in 1:1000
    
            #Determine the extinct species
            Np[findall(sum(Np,dims=2) .< 10),:] .= 0
            Ngen[findall(sum(Np,dims=2).==0),:] .= 0

            if all(sum(Np,dims=2) ==0) 
                break
            else

                newgen=zeros(Float64,nsp,nt)

                #Reproduction event
                for i in 1:size(Ngen)[1]

                    probs=Ngen[i,:]*Ngen[i,:]'

                    for j in 1:size(R)[3]
                        newgen[i,j]=sum(probs.*R[:,:,j])
                    end
                end

                newp=newgen .* sum(Np,dims=2)

                #Selection event

                for i in 1:size(newp)[1], j in 1:size(newp)[2]

                    comps=sum(A1[j,:] .* newp[1:end .!=i,:]') + sum(A0[j,:] .* newp[i,:]) 
                    Np[i,j]=newp[i,j]*r[i]*(1-comps)
                    #Np[i,j]=newp[i,j]+(newp[i,j]*(1-comps))

                end

                Np[findall(Np .<1)] .= 0
                Ngen= Np ./ sum(Np,dims=2)
                Ngen[isnan.(Ngen)].=0
            end

            if m ∈ samples

                dat1=DataFrame(nloci=n,reps=rep,omega=omega,t=t,a1=a1,kernel=i4,time=m,sp=1:nsp,
                                trmean=sum(eachcol(geno' .*Ngen)),pop=vec(sum(Np,dims=2)))

                append!(result,dat1)
            end

        end 
    end
            CSV.write(string("compSK_",rep,"_",n,"_",".csv"),result)

    end
    
end



In [ ]:
pmap(compsim,pars)

In [5]:
using CSV, DataFrames, Random, LinearAlgebra, Distances, Distributions, SpecialFunctions, Plots, Distributed

addprocs(11)

@everywhere begin
    
using CSV, DataFrames, Random, LinearAlgebra, Distances, Distributions, SpecialFunctions, Plots
    
nsp=20

samples=[collect(0:10:100); collect(100:50:1000)]

#Necessary functions
    
 #Necessary functions

    function alpha_gt(x::Float64, y::Float64, omega::Float64, t::Float64)
    
        al=0
        if(abs(x-y)<t) 
            
             al=exp(-((x-y)^2)/(omega^2))

        end
        return al
    end
    
    function alpha_tri(x::Float64, y::Float64, slope::Float64, t::Float64)
        
        alpha=0.0
        
        if ( abs(x-y) < t) 
            
            alpha=1- (slope*abs(x-y))
            
        end
            
        return max(0,alpha)
        
    end
    
    function mnnds(a::Vector{Float64})
    
    a=filter!(!isnan,a)
    if(length(a)>1)
        a=sort(a,rev=true)
        nnd=zeros(Float64,length(a))
        nnd[1]=a[2]-a[1]
        nnd[length(nnd)]=a[length(a)]-a[length(a)-1]
        for i in 2:((length(a)-1))
            nnd[i]=min((a[i-1]-a[i]),(a[i]-a[i+1]))
        end
        ranges=a[1]-a[end]
        mnnd=sum(nnd)/length(nnd)
        mmax=ranges/(length(a)-1)

        return mnnd/mmax
        
        else return 0.0
        end
        
    end
        
    
    function qgprob(n::Int64)
    
        #All possible phenotypes
        pheno= collect(1:(2*n+1)) ./ (2*n+1)
        nt=length(pheno)
    
        G=zeros(Float64,n+1,n+1,n+1)
    
        for i in 0:n, j in 0:i, k in max(0,(i+j-n)):min(n,(i+j))
                    m=collect(0:min(j,k,i+j-k))
                    G[1+i,1+j,1+k]=sum(pdf.(Hypergeometric(i,n-i,j),m).*pdf.(Binomial.(i+j .- (2 .* m)),k .- m))
        end
    
        for k in 0:n
            G[:,:,1+k]=G[:,:,1+k]+transpose(G[:,:,1+k])
            for i1 in 0:n
                G[i1+1,i1+1,k+1] /= 2
            end
        end
    
        ind_haplR=zeros(Float64,2*n+1, 2*n+1)
    
        for k in 0:n
            for i in 0:n
                 ind_haplR[1+i,1+k] = G[1+i,1,1+k]
                for j in 0:n
                    ind_haplR[1+j+n,1+k]=G[1+n,1+j,1+k]
                end
            end
        end
    
        R=zeros(Float64,nt,nt,nt)
    
        for i in 0:(2*n), j in 0:(2*n), q in 0:(2*n)
             R[1+i,1+j,1+q]= sum(ind_haplR[1+i,1 .+ (0:q)] .*
                                 ind_haplR[1+j,1+q .- (0:q)])
        end
    
        return R
    end
    
    
     function single_sim1(time,r,K1,K2,a1,A,R,Ng0,Npop)
        
            nsp=size(Ng0)[1]
            nt=size(Ng0)[2]
            
            Np0=Ng0 .*Npop
            Ngen=deepcopy(Ng0)
            Np=deepcopy(Np0)
        
            dat=zeros(Float64,time+1,nsp,nt)
    
            dat[1,:,:]=Np
            
            #Start the simulation
            for m in 2:(time+1)
        
                #Determine the extinct species
                Np[findall(sum(Np,dims=2) .< 10),:] .= 0
                Ngen[findall(sum(Np,dims=2) .==0),:] .= 0
    
                if all(sum(Np,dims=2) ==0) 
                    break
                else
    
                    newgen=zeros(Float64,nsp,nt)
    
                    #Reproduction event
                    for i in findall(!iszero,sum(eachcol(Ngen)))
    
                        probs=Ngen[i,:]*Ngen[i,:]'
                    
                        newgen[i,:]=[sum(probs.*R[:,:,x1]) for x1 in 1:nt]
                    
                        newgen[i,:] ./= sum(newgen[i,:])
                        
                    end
    
                    newgen .*= sum(Np,dims=2)
    
                    #Selection event
                
                    if size(A)[1]>1
    
                        for i1 in 1:size(newgen)[1]
                    
                            #This is where we have to make big choices!! a)Whether or not to make conspecific individuals compete the 
                            #same way as heterospecifics. b) Interprete the meaning of carrying capacity. Do different numbers
                            #apply for different species? If yes, what is the interpretation of this type of competition?
                           
                            #Reduction in growth rate due to intraspecific competition (not trait-dependent!). The carrying capacity
                            #here is assumed to be the same for all species.

                            rdash=r[i1] * (1 -(sum(newgen[i1,:])/K2))
                        
                            #Impact of interspecific competition

                            comps=[(a1.*sum((A[x2,:]) .* newgen[1:end .!=i1,:]')) for x2 in 1:nt]
                        
                            Np[i1,:] = newgen[i1,:] + (newgen[i1,:] .* rdash .* (1 .-(comps ./K1)))
                        
                        end
                    
                    end
    
                    Np[findall(Np .<1)] .= 0
                    Ngen= Np ./ sum(Np,dims=2)
                    Ngen[isnan.(Ngen)].=0
                end
                
            dat[m,:,:]=Np
                
            end
        
        return dat
        
    end
    
    
    #Blanket function to create trait and population trajectories
    function getsum(time,r,K1,K2,a1,A,R,Ng0,Npop)
        
        res=single_sim1(time,r,K1,K2,a1,A,R,Ng0,Npop)
        
        tstep=size(res)[1]
        nsp=size(res)[2]
        nt=size(res)[3]
        

        pops=zeros(Float64,tstep,nsp)
        trmeans=zeros(Float64,tstep,nsp)

        for i in 1:nsp

        pops[:,i]=[sum(res[x,i,:]) for x in 1:tstep]
        trmeans[:,i]=[sum(res[x,i,:] .* geno)/sum(res[x,i,:])  for x in 1:tstep]

        end

        return([pops,trmeans])
        
    end
        
#parameter combinations

reps=collect(1:10)
loci=[2,3,4,5,6,7,8,10,15,20,50]

pars=collect(Iterators.product(reps,loci))

function compsim(par::Tuple{Int64, Int64})
    
    rep=par[1]
    n=par[2]

    geno= collect(range(-1,stop=1,length=2*n+1))
    nt=length(geno)
    
     N0=zeros(Float64,nsp,nt)
     rands=rand(Uniform(-0.6,0.6),nsp)
     [N0[i,:]=pdf.(truncated(Normal(rands[i],0.2),-1.0,1.0),geno) for i in 1:nsp]

    N0=N0 ./ sum(N0,dims=2)

    r=abs.(rand(Uniform(1,1.1),nsp))
        
    #Set up a probability matrix of an offspring having a phenotype x when the parents have the phenotypes v and w resp.
    # This code follows the diploid version of an exact hypergeometric model
    #from Shpak and Kondrashov (1999)
    haplR=zeros(Float64,n+1,n+1,n+1)

    for i in 0:n, j in 0:i, k in 0:min(n,(i+j))
        haplR[1+i,1+j,1+k]=sum(pdf.(Hypergeometric(i,n-i,j),max(0, i+j-n):min(i, j)) .*
                              map(x->pdf(Binomial(i+j-2*x),k-x),collect(max(0, i+j-n):min(i,j))))
    end

    for k in 0:n
        haplR[:,:,1+k]=haplR[:,:,1+k]+transpose(haplR[:,:,1+k])
        for i1 in 0:n
            haplR[i1+1,i1+1,k+1] /= 2
        end
    end

    ind_haplR=zeros(Float64,2*n+1, 2*n+1)

    for k in 0:n
        for i in 0:n
            ind_haplR[1+i,1+k] = haplR[1+i,1,1+k]
            for j in 0:n
                ind_haplR[1+j+n,1+k]=haplR[1+n,1+j,1+k]
            end
        end
    end

    R=zeros(Float64,nt,nt,nt)

    for i in 0:(2*n), j in 0:(2*n), q in 0:(2*n)
        R[1+i,1+j,1+q]= sum(ind_haplR[1+i,1 .+ (0:q)] .* 
                            ind_haplR[1+j,1+q .- (0:q)])
    end
    
    
    omegas=[0.1,0.5]
    ts=[0.2,0.5,2]
    a1s=[0.1,0.5,1]
    kernel=["Gaussian","Triangle"]
    K1=1000
    K2=5000
    
    result=DataFrame()
        
    for i1 in omegas, i2 in ts, i3 in a1s, i4 in kernel
        
        omega=i1
        t=i2
        a1=i3
        
        #Pre-calculate coefficients of competition between pairs of genotypes

        A=zeros(Float64,nt,nt)
        
        for j1 in 1:nt, j2 in 1:nt
                
            if i4=="Gaussian"
    
            A[j1,j2]=alpha_gt(geno[j2],geno[j1],omega,t)
                    
                elseif i4=="Triangle"
                    
            A[j1,j2]=alpha_tri(geno[j2],geno[j1],omega,t)
                end
        end
                    
        #Start the simulation
        res1=getsum(2000,r,K1,K2,0.1,A,R,N0,1000.0)
        trdat=res1[2]
        mnnddat=[mnnds(trdat[x,:]) for x in 1:2001]
        
        append!(res,DataFrame(nloci=n,reps=rep,a1s=a1,K1s=K1,kernel=p2,traits=p4,time=1:2001,mnnd=mnnddat))


    end
            CSV.write(string("compsim_",rep,"_",n,"_",".csv"),result)

    end
    
end



In [6]:
pmap(compsim,pars)

LoadError: On worker 2:
UndefVarError: `geno` not defined
Stacktrace:
  [1] [0m[1m#34[22m
[90m    @[39m [90m.\[39m[90m[4mnone:0[24m[39m
  [2] [0m[1miterate[22m
[90m    @[39m [90m.\[39m[90m[4mgenerator.jl:47[24m[39m[90m [inlined][39m
  [3] [0m[1mcollect[22m
[90m    @[39m [90m.\[39m[90m[4marray.jl:782[24m[39m
  [4] [0m[1mgetsum[22m
[90m    @[39m [90m.\[39m[90m[4mIn[5]:201[24m[39m
  [5] [0m[1mcompsim[22m
[90m    @[39m [90m.\[39m[90m[4mIn[5]:300[24m[39m
  [6] [0m[1m#invokelatest#2[22m
[90m    @[39m [90m.\[39m[90m[4messentials.jl:816[24m[39m
  [7] [0m[1minvokelatest[22m
[90m    @[39m [90m.\[39m[90m[4messentials.jl:813[24m[39m
  [8] [0m[1m#110[22m
[90m    @[39m [90mC:\Users\mihir\AppData\Local\Programs\Julia-1.9.2\share\julia\stdlib\v1.9\Distributed\src\[39m[90m[4mprocess_messages.jl:285[24m[39m
  [9] [0m[1mrun_work_thunk[22m
[90m    @[39m [90mC:\Users\mihir\AppData\Local\Programs\Julia-1.9.2\shar